In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
products = pd.read_csv("data/products.csv")
store_purchases = pd.read_csv("data/store_purchases.csv")
users = pd.read_csv("data/users.csv")
web_visits = pd.read_csv("data/web_visits.csv")

In [3]:
store_user = store_purchases.merge(users, how="left", left_on = ["user_id"], right_on = ["user_ID"])
store_user.drop(["user_ID"], inplace=True, axis=1)
store_user

,user_id,date,family,class,type,product,product_id,price,card_owner
0,49263,2020-02-26,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
1,30940,2022-12-16,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
2,11386,2020-02-18,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
3,37749,2020-11-04,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
4,14128,2021-02-10,family_6,class_3,type_4,product_3,00043a7a-ba01-4653-a8c9-dfc1c256372d,5.0,False
...,...,...,...,...,...,...,...,...,...
171648,34856,2021-08-08,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171649,47325,2022-10-21,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171650,30303,2022-05-12,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False
171651,20029,2021-01-17,family_6,class_1,type_1,product_12,6bcfe8b4-42c2-4089-bccc-b670ad8210d6,152.0,False


In [4]:
df = web_visits.merge(store_user[["user_id", "product_id", "date", "card_owner"]], how = "left", on=["user_id", "product_id"])
df

,family,class,type,web_visit_date,user_id,product,price,product_id,conversion,date,card_owner
0,family_9,class_8,type_2,2022-03-02,1770.0,product_17,103,567e07ef-45e3-406b-a2b1-1175187504be,0,NaN,NaN
1,family_4,class_8,type_2,2022-04-08,43333.0,product_30,1,5f7dcb42-5eb4-4aaf-bd36-40e766fc7449,0,NaN,NaN
2,family_9,class_8,type_1,2022-03-14,33025.0,product_16,2,6d2ac9e4-9baf-4338-b3d9-671bcdcb36eb,0,NaN,NaN
3,family_4,class_8,type_2,2022-04-25,24377.0,product_22,34,717b0355-71aa-4be6-9eff-afa5c7ec95c7,0,NaN,NaN
4,family_4,class_8,type_2,2022-09-19,40274.0,product_32,1,75b6a327-af04-413a-a605-a07cc66b3fb4,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
27238191,family_2,class_1,type_1,2020-06-17,18550.0,product_19,163,1b002490-5f32-4482-8a50-83379e511519,0,NaN,NaN
27238192,family_2,class_1,type_1,2020-06-13,27973.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN
27238193,family_2,class_1,type_1,2022-02-07,39666.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN
27238194,family_5,class_1,type_2,2022-05-09,28123.0,product_5,166,21c5eef8-2ffe-4dbf-9b9a-04f87715e6af,0,2021-12-06,False


In [5]:
df["offline_conversion"] = 0

In [6]:
df_converted = df[df["conversion"]==0]
df_converted

,family,class,type,web_visit_date,user_id,product,price,product_id,conversion,date,card_owner,offline_conversion
0,family_9,class_8,type_2,2022-03-02,1770.0,product_17,103,567e07ef-45e3-406b-a2b1-1175187504be,0,NaN,NaN,0
1,family_4,class_8,type_2,2022-04-08,43333.0,product_30,1,5f7dcb42-5eb4-4aaf-bd36-40e766fc7449,0,NaN,NaN,0
2,family_9,class_8,type_1,2022-03-14,33025.0,product_16,2,6d2ac9e4-9baf-4338-b3d9-671bcdcb36eb,0,NaN,NaN,0
3,family_4,class_8,type_2,2022-04-25,24377.0,product_22,34,717b0355-71aa-4be6-9eff-afa5c7ec95c7,0,NaN,NaN,0
4,family_4,class_8,type_2,2022-09-19,40274.0,product_32,1,75b6a327-af04-413a-a605-a07cc66b3fb4,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27238191,family_2,class_1,type_1,2020-06-17,18550.0,product_19,163,1b002490-5f32-4482-8a50-83379e511519,0,NaN,NaN,0
27238192,family_2,class_1,type_1,2020-06-13,27973.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,0
27238193,family_2,class_1,type_1,2022-02-07,39666.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,0
27238194,family_5,class_1,type_2,2022-05-09,28123.0,product_5,166,21c5eef8-2ffe-4dbf-9b9a-04f87715e6af,0,2021-12-06,False,0


In [7]:
df_converted["offline_conversion"] = df_converted["offline_conversion"].where(df_converted["date"].isna(), 1)
df_converted

C:\Users\Rehan Ibrahim\AppData\Local\Temp\ipykernel_13940\1083755211.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_converted["offline_conversion"] = df_converted["offline_conversion"].where(df_converted["date"].isna(), 1)


,family,class,type,web_visit_date,user_id,product,price,product_id,conversion,date,card_owner,offline_conversion
0,family_9,class_8,type_2,2022-03-02,1770.0,product_17,103,567e07ef-45e3-406b-a2b1-1175187504be,0,NaN,NaN,0
1,family_4,class_8,type_2,2022-04-08,43333.0,product_30,1,5f7dcb42-5eb4-4aaf-bd36-40e766fc7449,0,NaN,NaN,0
2,family_9,class_8,type_1,2022-03-14,33025.0,product_16,2,6d2ac9e4-9baf-4338-b3d9-671bcdcb36eb,0,NaN,NaN,0
3,family_4,class_8,type_2,2022-04-25,24377.0,product_22,34,717b0355-71aa-4be6-9eff-afa5c7ec95c7,0,NaN,NaN,0
4,family_4,class_8,type_2,2022-09-19,40274.0,product_32,1,75b6a327-af04-413a-a605-a07cc66b3fb4,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...
27238191,family_2,class_1,type_1,2020-06-17,18550.0,product_19,163,1b002490-5f32-4482-8a50-83379e511519,0,NaN,NaN,0
27238192,family_2,class_1,type_1,2020-06-13,27973.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,0
27238193,family_2,class_1,type_1,2022-02-07,39666.0,product_16,1,20b90b1a-beeb-4308-82ba-7c55121141d8,0,NaN,NaN,0
27238194,family_5,class_1,type_2,2022-05-09,28123.0,product_5,166,21c5eef8-2ffe-4dbf-9b9a-04f87715e6af,0,2021-12-06,False,1


In [8]:
df_converted["web_visit_date"] = pd.to_datetime(df_converted["web_visit_date"])

C:\Users\Rehan Ibrahim\AppData\Local\Temp\ipykernel_13940\768794643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_converted["web_visit_date"] = pd.to_datetime(df_converted["web_visit_date"])


In [9]:
df_converted["date"] = pd.to_datetime(df_converted["date"])

C:\Users\Rehan Ibrahim\AppData\Local\Temp\ipykernel_13940\3661899603.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_converted["date"] = pd.to_datetime(df_converted["date"])


In [10]:
data = df_converted.copy()

In [11]:
X = data.drop('offline_conversion', axis=1)
y = data['offline_conversion']

In [31]:
#X.drop(columns=['user_id'],inplace=True)
X.head()

,price,card_owner
0,103,NaN
1,1,NaN
2,2,NaN
3,34,NaN
4,1,NaN


In [32]:
from catboost import CatBoostClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, task_type="GPU", random_seed=42)
model.fit(X,y)
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(report)
print('Confusion Matrix:')
print(confusion)
feature_importance = model.get_feature_importance()
print('Feature Importance:')
for i, importance in enumerate(feature_importance):
    print(f'Feature {i}: {importance}')


0:	learn: 0.5979078	total: 240ms	remaining: 23.8s
1:	learn: 0.5204491	total: 365ms	remaining: 17.9s
2:	learn: 0.4553879	total: 524ms	remaining: 16.9s
3:	learn: 0.4004800	total: 649ms	remaining: 15.6s
4:	learn: 0.3536216	total: 775ms	remaining: 14.7s
5:	learn: 0.3134522	total: 900ms	remaining: 14.1s
6:	learn: 0.2783457	total: 1.03s	remaining: 13.6s
7:	learn: 0.2479040	total: 1.22s	remaining: 14s
8:	learn: 0.2211516	total: 1.34s	remaining: 13.6s
9:	learn: 0.1975747	total: 1.46s	remaining: 13.2s
10:	learn: 0.1767610	total: 1.62s	remaining: 13.1s
11:	learn: 0.1583966	total: 1.75s	remaining: 12.8s
12:	learn: 0.1420778	total: 1.93s	remaining: 12.9s
13:	learn: 0.1275106	total: 2.12s	remaining: 13s
14:	learn: 0.1145789	total: 2.24s	remaining: 12.7s
15:	learn: 0.1030291	total: 2.43s	remaining: 12.8s
16:	learn: 0.0927482	total: 2.59s	remaining: 12.7s
17:	learn: 0.0834895	total: 2.78s	remaining: 12.7s
18:	learn: 0.0751948	total: 2.9s	remaining: 12.4s
19:	learn: 0.0677507	total: 3.03s	remaining: 1

In [33]:
model.score(X_test,y_test)

1.0

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Step 4: XGBoost Model Initialization
model = xgb.XGBClassifier(n_estimators=100, tree_method="gpu_hist",  max_depth=3, learning_rate=0.1, random_state=42)

# Step 6: Model Training
model.fit(X_train, y_train)

# Step 7: Model Evaluation
val_predictions = model.predict(X_val)
accuracy = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy: {accuracy}')

# Step 10: Final Model Evaluation
test_predictions = model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy}')

In [ ]:
from matplotlib import pyplot as plt
feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(12, 6))
plt.barh(range(len(sorted_idx)), feature_importance[sorted_idx], align='center')
plt.yticks(range(len(sorted_idx)), np.array(X_test.columns)[sorted_idx])
plt.title('Feature Importance')